In [1]:
import sys
import re
import pandas as pd
from collections import defaultdict
from glob import glob
import logLibrary as lib
import os

In [2]:
REGEX = 'Regex'
DESC = 'Description'
REF = 'Reference'
CAUSE = 'Cause'
IMPACT = 'Impact'
exceptions = defaultdict(int)
errorLogs = defaultdict(int)

RuleIDs = None
#configDict.get('serviceName').split(",")

In [3]:
def registerKeyValue(key,value,configDict):
    configDict[key] += value

In [4]:
def addSignatureDetails(rule):
    rule += '.'
    details = "%s: %s,\n %s: %s,\n %s:%s,\n %s:%s" %(DESC, configDict.get(rule+DESC), REF, configDict.get(rule+REF), CAUSE, configDict.get(rule+CAUSE), IMPACT, configDict.get(rule+IMPACT))
    return str(details) 

In [5]:
def loadInputFile(fileName):
    configDict = defaultdict(str)
    global RuleIDs
    with open(fileName, "r") as fh:
        data = fh.read()
    pattern = re.compile(r'(\w+.\w+)[\s]*=[\s]*(.*)', re.MULTILINE)
    matches = [m.groups() for m in pattern.finditer(data)]
    for m in matches:
        registerKeyValue(m[0],m[1],configDict)
    data = None
    RuleIDs = configDict.get('RuleIDs').split(",")
    return configDict

In [6]:
def matchSignatures(logs,configDict):
    signature = ""
    for ruleid in RuleIDs:
        matched = re.search(configDict.get(ruleid+'.'+REGEX), logs, re.DOTALL|re.MULTILINE)
        if not matched:
            signature = 'NO SIGNATURE DEFINITION FOUND!!!'
            continue
        else:
            signature = addSignatureDetails(ruleid)
            matched = None
            break
    return signature

In [9]:
SignatureFile = r"C:\CodeAnalytics\Script\signatures.txt"
folder = glob('C:/CodeAnalytics/Data/Output/'+"*.csv")
for file in folder:
    filename = os.path.split(file)[1].split('.')[0]
    print(filename)
    configDict = loadInputFile(SignatureFile)
    df = pd.read_csv(file)
    df['Description'] = df['Exception'].apply(lambda x:matchSignatures(x,configDict))
    #print(df)
    df.to_csv('C:/CodeAnalytics/Data/Output/'+filename+".csv")
    

POD1_Rest_Processed
POD2_Rest_Processed


In [ ]:
outputpath = "../Data/Output/

In [9]:
#main(SignatureFile,file)

NameError: name 'df' is not defined

In [7]:
# def main(SignatureFile,CsvFile):
#     loadInputFile(SignatureFile)
#     df = pd.read_csv(CsvFile)
#     df['Description'] = df['Exception'].apply(str).apply(lambda x:matchSignatures(x))
#     df.to_csv(Description.csv")
#     #return df['Description']

In [8]:
#loadInputFile(r"C:\sunil\python files\signatures.txt")

In [9]:
# file = r"C:\CodeAnalytics\Data\Output\POD2_Rest_Processed.csv"
# df = pd.read_csv(file)
# df['Description'] = df['Exception'].apply(str).apply(lambda x:matchSignatures(x))
# df.to_csv(r"C:\sunil\csv files\Description.csv")

In [ ]:
SignatureFile = r"C:\CodeAnalytics\Script\signatures.txt"
json_obj = lib.json_reader(r"C:\CodeAnalytics\Config\configpath.json")
DataPath = json_obj['DataPath']
outputpath = DataPath+'Output/'
print('outputpath',outputpath)
folder = glob(outputpath+'*.csv')
for file in folder:
    filename = os.path.split(file)[1].split('.')[0]
    print(filename)
    loadInputFile(SignatureFile)
    df = pd.read_csv(file)
    df['Description'] = df['Exception'].apply(str).apply(lambda x:matchSignatures(x))
    print(df)
    df.to_csv(outputpath+filename+".csv")